In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import SkyCoord as coord
import astropy.units as u
from astropy.coordinates import Angle
import warnings
warnings.filterwarnings('ignore')

###  Step 1:  Importing gamma ray burst data

<font color = 'purple'>Data I have used for gamma ray burst was taken form this site below.</font>

https://swift.gsfc.nasa.gov/results/batgrbcat/summary_cflux/summary_general_info/summary_general.txt


In [2]:
column_names = ['GRBname', 'RA_ground', 'DEC_ground']
df_grb = pd.read_csv('/home/astroguy/frb_summerresearch/grb_data_copy.csv', delimiter='|', header = 0)
df_grb.tail(1)

,GRBname,Trig_ID,Trig_time_met,Trig_time_UTC,RA_ground,DEC_ground,Image_position_err,Image_SNR,T90,T90_err,T50,T50_err,Evt_start_sincetrig,Evt_stop_sincetrig,pcode,Trigger_method,XRT_detection,"comment,,,"
1390,GRB041217,100116,124961305.236,2004-12-17T07:28:25.236000,1.647929e+02,-1.794047e+01,1.374749e+00,1.930439e+01,5.668000e+00,7.297123e-01,2.700000e+00,2.512051e-01,-2,18,0.2969,rate trigger,No,",,,"


<font color = 'purple'>Note:  According to the description in the catalog RA_ground and DEC_ground both has been measured in the unit of degree.</font> 

In [6]:
# Taking only necessary subset of the whold data from the catalog

df_grb_subset = df_grb.iloc[:,[0,4,5]]
print(df_grb_subset.shape)
df_grb_subset.tail(2)

(1391, 3)


,GRBname,RA_ground,DEC_ground
1389,GRB041219A,N/A,N/A
1390,GRB041217,1.647929e+02,-1.794047e+01


In [8]:
# Since some data are missing we want to get rid of those rows. Using for loop to pick the rows with no data.


empty_row_num = []
for num in np.arange(0, len(df_grb_subset.iloc[:])):
    if df_grb_subset.iloc[num, 1] == ' N/A ' or df_grb_subset.iloc[num, 2] == ' N/A ':
        empty_row.append(num)

empty_row_num

[]

In [12]:
# Using drop function to drop all the rows with no data. Ten rows with no data have been removed.

df_grb_subset.drop(empty_row_num, axis = 0, inplace = True)
print(df_grb_subset.shape)

(1381, 3)


###  Step 2: Working with fast radio burst data

<font color = 'purple'>Data taken from link below</font>

https://www.herta-experiment.org/frbstats/catalogue

In [13]:
columns = ['ra', 'dec', 'dm', 'flux', 'redshift', 'frb', 'l', 'b', 'width', 'fluence']
df_herta = pd.read_csv('herta_frb_catalogue.csv', usecols=columns).replace('-', '0')
df_herta.head(1)

,frb,ra,dec,l,b,dm,flux,width,fluence,redshift
0,FRB 20010125A,19:06:53,-40:37:14,356.64,-20.02,790.3,0.54,0,0,0.7083


<font color = 'purple'>Unit of RA in the catalog is hour:min:sec and that of declination is in deg:arcmin:arcsec</font>

In [14]:
herta_subset = df_herta.iloc[:, [0, 1,2]]
herta_subset.head(1)

,frb,ra,dec
0,FRB 20010125A,19:06:53,-40:37:14


<font color = 'purple'>We convert RA and DEC to the unit of degree to make it uniform with the unit of RA and DEC in GRB data. We use astropy library to convert coordinates to degress. </font>

Reference:  https://docs.astropy.org/en/stable/coordinates/angles.html

<font color = 'purple'>This site maintained by penstate can be used as checker to see if our conversion has been done correctly.</font>

https://www.swift.psu.edu/secure/toop/convert.htm

In [15]:
RA_in_degrees  = []
DEC_in_degrees = []

for k in np.arange(0, len(herta_subset['frb'])):
    RA_degree = Angle(herta_subset['ra'][k], unit = 'hourangle')   # RA is in  hr:min:sec
    DEC_degree = Angle(herta_subset['dec'][k], unit = 'degree')    # DEC is in deg:arcmin:arcsec
    RA_in_degrees.append(RA_degree.degree)
    DEC_in_degrees.append(DEC_degree.degree)
    
herta_subset['RA_in_deg'] = RA_in_degrees
herta_subset['DEC_in_deg'] = DEC_in_degrees

herta_subset.head(1)

,frb,ra,dec,RA_in_deg,DEC_in_deg
0,FRB 20010125A,19:06:53,-40:37:14,286.720833,-40.620556


In [19]:
# This function calculates distance using formula shown below.

def distance(ra_frb, ra_grb, dec_frb, dec_grb):
    diff_ra = ra_frb - ra_grb
    diff_dec = dec_frb - dec_grb
    avg_dec = (dec_frb + dec_grb)/2
    frb_grb_dist = np.sqrt((diff_ra*np.cos(avg_dec*(np.pi/180)))**2 + diff_dec**2)
    return frb_grb_dist

$$ dist = \sqrt{ ({\Delta\alpha\cos{\delta}})^2 + {\Delta{\delta}}^2} $$

where, 

<br>$ \Delta \alpha = ra_{1} - ra_{2} $
<br>$\Delta \delta = de_{1} - de_{2} $ 
<br>$ \delta = \frac{de_{1} + de_{2}}{2}$

In [17]:
dist_s_no = []
for m in np.arange(0, len(herta_subset['frb'])):
    frb_ra = herta_subset['RA_in_deg'][m]
    frb_dec = herta_subset['DEC_in_deg'][m]

    dist_list = []
    for n in np.arange(0, len(df_grb_subset.iloc[:])):
        grb_ra = float(df_grb_subset.iloc[n][1])
        grb_dec = float(df_grb_subset.iloc[n][2])
        
        dist = distance(frb_ra, grb_ra, frb_dec, grb_dec)
        dist_list.append(dist)
        
    min_dist_val = min(dist_list)
    grb_index = dist_list.index(min_dist_val)
    dist_s_no.append([m, grb_index, min_dist_val])
    
